<a href="https://colab.research.google.com/github/agungfirdaus717-ux/torentotgd/blob/main/SRTBingtrranslate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SRT Bing Translate Indonesia

In [ ]:
# === Instalasi Library ===
!pip install translators==5.8.3 pysrt

import translators as ts
import pysrt
from google.colab import files
import math, sys

# === Upload File SRT ===
print("📂 Silakan upload file subtitle (.srt)")
uploaded = files.upload()
input_file = list(uploaded.keys())[0]

# === Baca File SRT ===
subs = pysrt.open(input_file)
total_lines = len(subs)
print(f"📑 Total baris subtitle: {total_lines}")

# === Konfigurasi ===
SPLIT_TOKEN = " <<<###SPLIT###>>> "  # separator aman (pakai spasi)
MAX_CHARS = 3000   # batas aman karakter per request
BATCH_SIZE = 50    # coba per 50 baris, auto split kalau terlalu panjang

# === Fungsi bagi batch ===
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

# === Fungsi translate dengan auto split & validasi ===
def translate_batch(batch):
    texts = [sub.text for sub in batch if sub.text.strip()]
    if not texts:
        return []

    joined_text = SPLIT_TOKEN.join(texts)

    # kalau terlalu panjang → pecah jadi setengah
    if len(joined_text) > MAX_CHARS and len(batch) > 1:
        mid = len(batch) // 2
        return (
            translate_batch(batch[:mid]) +
            translate_batch(batch[mid:])
        )

    try:
        translated = ts.translate_text(
            joined_text, translator='bing', to_language='id'
        )
        parts = translated.split(SPLIT_TOKEN)

        # === Validasi jumlah hasil ===
        if len(parts) != len(texts):
            results = []
            for sub in batch:
                try:
                    results.append(
                        ts.translate_text(sub.text, translator='bing', to_language='id')
                    )
                except:
                    results.append(sub.text)
            return results
        return parts

    except Exception as e:
        if len(batch) > 1:
            mid = len(batch) // 2
            return (
                translate_batch(batch[:mid]) +
                translate_batch(batch[mid:])
            )
        else:
            print(f"\n❌ Gagal translate baris {batch[0].index}: {e}")
            return [batch[0].text]

# === Translate semua subtitle dengan progress inline ===
print("🚀 Proses terjemahan ke Bahasa Indonesia...\n")

processed = 0
for batch_idx, batch in enumerate(chunks(list(subs), BATCH_SIZE), start=1):
    translated_lines = translate_batch(batch)
    for sub, trans_text in zip(batch, translated_lines):
        sub.text = trans_text

    processed = min(batch_idx * BATCH_SIZE, total_lines)
    percent = (processed / total_lines) * 100
    sys.stdout.write(f"\r✅ Progress: {percent:.1f}% ({processed}/{total_lines})")
    sys.stdout.flush()

print("\n🎉 Terjemahan selesai!")

# === Simpan File Baru ===
output_file = "translated_" + input_file
subs.save(output_file, encoding="utf-8")

# === Download File ===
files.download(output_file)
print(f"📂 File hasil: {output_file}")
